# Classifier

http://www.cnblogs.com/rongyux/p/5645131.html

https://www.reddit.com/r/AskReddit/comments/4qfh01/what_are_some_classes_you_must_take_in/

# 引言
本程序是一个完整的机器学习过程，先编写基于python的爬虫脚本，爬取目标论坛网站的评论到本地存储，然后使用贝叶斯分类模型对评论进行分类，预测新的评论是否为垃圾评论。如果遇到大数据量的问题，可以把贝叶斯算法写成mapreduce模式，map负责把数据集划分成键值对格式，类序号为key，属 性向量为value，reduce进行汇总每类的先验概率和条件概率，主server汇总所有类的统计量。

In [33]:
import requests
from bs4 import BeautifulSoup
import re
import json
import time

# 爬虫

1 编写爬虫脚本，爬取目标论坛的评论。其中，headers是必须的，因为我们需要伪装成浏览器在访问论坛的服务器。使用requests包获取指定url的数据流。

In [10]:
headers = {
'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
'Accept-Language': 'zh-CN,zh;q=0.8,en-US;q=0.5,en;q=0.3',
'Accept-Encoding': 'gzip, deflate, br',
'Connection': 'keep-alive',
'Cookie': '__cfduid=d653bf931cbde10f9243b63e991f70dc41466778585; loid=a5WUnHRHlleKL9OSSR; loidcreated=2016-06-24T14%3A29%3A45.413Z; _recent_srs=t5_2qu49; _ga=GA1.2.54465388.1466778724; pc=ne; __utma=55650728.54465388.1466778724.1466778728.1466843492.2; __utmz=55650728.1466778728.1.1.utmcsr=(direct)|utmccn=(direct)|utmcmd=(none); __utmb=55650728.0.10.1466843492; __utmc=55650728',
'Host': 'www.reddit.com',
'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; WOW64; rv:47.0) Gecko/20100101 Firefox/47.0',
}

url = 'https://www.reddit.com/r/AskReddit/comments/4qfh01/what_are_some_classes_you_must_take_in/'
r = requests.get(url,headers=headers)
r.encoding = r.apparent_encoding

2 使用BeautifulSoup解析爬去的html文件，css定位我们需要的字段，输出到本地文件comments.txt保存即可。

In [43]:
soup = BeautifulSoup(r.text)
res = soup.select("div.md")
comments = []
for item1 in res[1:]:
    comments.append(item1.contents)
# print(comments)

fd = open('comments.txt','w+')
p_soup = BeautifulSoup(str(comments))
res2 = p_soup.findAll('p')
for item2 in res2:
    ct = str(item2.contents)
    print(ct[2:-2])
    fd.write(ct[2:-2] + '\n')
fd.close()

D:\Anaconda3\lib\site-packages\bs4\__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file D:\Anaconda3\lib\runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))


[[
Underwater Basket Weaving 101
Most colleges have required courses that you need to take depending on your major or other various factors. 
It differs between colleges, some of my friends go to different school and they have to take classes ive never heard of. Also depends on the major of the student
No matter what your major is, you should try to take a personal finance class. It teaches you really valuable skills that translate across majors, like the basics of investing your income and saving for retirement.
[deleted]
I wish
It depends on the institution and your major. Typically in my state an english, math, gender studies, and history class is all needed. Also a basic science course. We are also now mandated to take an online course in sexual assault and consent and I am interested to see how that plays out
What state requires gender studies?
Wisconsin. At least for their state schools which are typically more liberal arts than technical
I go to a Jesuit University in the Philip

# 三 实战1 -文本分类（应用过滤恶意留言等）

下面是二分类问题，文档只能属于0和1两个类别，

## 1 载入数据集：本地读取文件comments.txt中爬虫爬取的评论。

> 在爬虫爬取评论之后，需要人为对每条评论是否是恶意留言进行标注

In [51]:
from numpy import *
def loadDataSet():
    fd = open('comments.txt')
    postingList = []
    classVec = []
    for line in fd.readlines():
        print(line)
        tmp = line.split()
        postingList.append(tmp[1:])
        classVec.append(int(tmp[0]))
    return postingList,classVec

postingList,classVec = loadDataSet()

1 Underwater Basket Weaving 010

0 Most colleges have required courses that you need to take depending on your major or other various factors. 

0 It differs between colleges, some of my friends go to different school and they have to take classes ive never heard of. Also depends on the major of the student

0 No matter what your major is, you should try to take a personal finance class. It teaches you really valuable skills that translate across majors, like the basics of investing your income and saving for retirement.

1 I wish

0 It depends on the institution and your major. Typically in my state an english, math, gender studies, and history class is all needed. Also a basic science course. We are also now mandated to take an online course in sexual assault and consent and I am interested to see how that plays out

1 What state requires gender studies?

0 Wisconsin. At least for their state schools which are typically more liberal arts than technical

0 I go to a Jesuit University 

## 2 创建词汇表：利用集合结构内元素的唯一性，创建一个包含所有词汇的词表。

In [36]:
def createVocabList(dataSet):
    vocabSet = set([])  #create empty set
    for document in dataSet:
        vocabSet = vocabSet | set(document) #union of the two sets
    return list(vocabSet)
vocabSet = createVocabList(postingList)

## 3 把输入文本根据词表转化为计算机可处理的01向量形式：

　　eq，测试文本1： ['love', 'my', 'dalmation']

　 　　　词汇表：['cute', 'love', 'help', 'garbage', 'quit', 'I', 'problems', 'is', 'park', 'stop', 'flea', 'dalmation', 'licks', 'food', 'not', 'him', 'buying', 'posting', 'has', 'worthless', 'ate', 'to', 'maybe', 'please', 'dog', 'how', 'stupid', 'so', 'take', 'mr', 'steak', 'my']

　　　　向量化结果：[0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1]

In [41]:
def setOfWords2Vec(vocabList, inputSet):
    returnVec = [0]*len(vocabList)
    for word in inputSet:
        if word in vocabList:
            returnVec[vocabList.index(word)] = 1
        else: print("the word: %s is not in my Vocabulary!" % word)
    return returnVec

testset =  ['love', 'my', 'dalmation']
returnVec = setOfWords2Vec(vocabSet, testset)

the word: love is not in my Vocabulary!
the word: dalmation is not in my Vocabulary!


## 4 训练模型：在训练样本中计算先验概率 p(Ci) 和 条件概率 p(x,y | Ci)，本实例有0和1两个类别，所以返回p(x,y | 0)，p(x,y | 1)和p(Ci)。

　　此处有两个改进的地方：

　　　　（1）若有的类别没有出现，其概率就是0，会十分影响分类器的性能。所以采取各类别默认1次累加，总类别（两类）次数2，这样不影响相对大小。

　　　　（2）若很小是数字相乘，则结果会更小，再四舍五入存在误差，而且会造成下溢出。采取取log，乘法变为加法，并且相对大小趋势不变。

In [52]:
def trainNB0(trainMatrix,trainCategory):
    numTrainDocs = len(trainMatrix)
    numWords = len(trainMatrix[0])
    pAbusive = sum(trainCategory)/float(numTrainDocs)
    p0Num = ones(numWords); p1Num = ones(numWords)      #change to ones() 
    p0Denom = 2.0; p1Denom = 2.0                        #change to 2.0
    for i in range(numTrainDocs):
        if trainCategory[i] == 1:          
            p1Num += trainMatrix[i]
            p1Denom += sum(trainMatrix[i])
        else:          
            p0Num += trainMatrix[i]
            p0Denom += sum(trainMatrix[i])
    p1Vect = log(p1Num/p1Denom)          #change to log()
    p0Vect = log(p0Num/p0Denom)          #change to log()
    return p0Vect,p1Vect,pAbusive

## 5 分类：根据计算后，哪个类别的概率大，则属于哪个类别。

In [26]:
def classifyNB(vec2Classify, p0Vec, p1Vec, pClass1):
    p1 = sum(vec2Classify * p1Vec) + log(pClass1)    #element-wise mult
    p0 = sum(vec2Classify * p0Vec) + log(1.0 - pClass1)
    if p1 > p0:
        return 1
    else: 
        return 0

## 6 测试函数

1. 加载数据集+提炼词表；
2. 训练模型：根据六条训练集计算先验概率和条件概率；
3. 测试模型：对训练两条测试文本进行分类。

In [49]:
def testingNB():
    listOPosts,listClasses = loadDataSet()
    myVocabList = createVocabList(listOPosts)
    trainMat=[]
    for postinDoc in listOPosts:
        trainMat.append(setOfWords2Vec(myVocabList, postinDoc))
    p0V,p1V,pAb = trainNB0(array(trainMat),array(listClasses))
    testEntry = ['friends', 'my', 'take']
    thisDoc = array(setOfWords2Vec(myVocabList, testEntry))
    print(testEntry,'classified as: ',classifyNB(thisDoc,p0V,p1V,pAb))
    testEntry = ['stupid', 'garbage']
    thisDoc = array(setOfWords2Vec(myVocabList, testEntry))
    print(testEntry,'classified as: ',classifyNB(thisDoc,p0V,p1V,pAb))
testingNB()

1 Underwater Basket Weaving 010

0 Most colleges have required courses that you need to take depending on your major or other various factors. 

0 It differs between colleges, some of my friends go to different school and they have to take classes ive never heard of. Also depends on the major of the student

0 No matter what your major is, you should try to take a personal finance class. It teaches you really valuable skills that translate across majors, like the basics of investing your income and saving for retirement.

1 I wish

0 It depends on the institution and your major. Typically in my state an english, math, gender studies, and history class is all needed. Also a basic science course. We are also now mandated to take an online course in sexual assault and consent and I am interested to see how that plays out

1 What state requires gender studies?

0 Wisconsin. At least for their state schools which are typically more liberal arts than technical

0 I go to a Jesuit University 

缺点：词表只能记录词汇是否出现，不能体现这个词汇出现的次数。改进方法：采用词袋模型，见下面垃圾邮件分类实战。

# 四 算法的MapReduce形式

> 本人正在把这个贝叶斯分类算法转换成分布式算法，初步思想是，可以把贝叶斯算法写成mapreduce模式，map负责把数据集划分成键值对格式，类序号为key，属 性向量为value，reduce进行汇总每类的先验概率和条件概率，主server汇总所有类的统计量。